In [4]:
import glob, json, pathlib
from tqdm.notebook import tqdm
sanitychecks = list(glob.glob(r"D:\ig_pipeline\cad\objects\*\artifacts\sanitycheck.json"))
jsons = {}
for x in tqdm(sanitychecks):
    with open(x, "r") as f:
        name = pathlib.Path(x).parts[-3]
        jsons[name] = json.load(f)

  0%|          | 0/1931 [00:00<?, ?it/s]

In [2]:
jsons

{'Beechwood_0_garden': {'success': False,
  'errors': ['bush-uyznuu-0 has negative object offset scale. Try mirroring + resetting scale.',
   'gate-vqjuvg-0-leaf2-base_link-R-upper has negative object offset scale. Try mirroring + resetting scale.',
   'L-sofa-rqipbs-0 has negative object offset scale. Try mirroring + resetting scale.',
   'bush-uyznuu-1 has negative object offset scale. Try mirroring + resetting scale.',
   'bush-uyznuu-2 has negative object offset scale. Try mirroring + resetting scale.',
   'bush-uyznuu-3 has negative object offset scale. Try mirroring + resetting scale.',
   'bush-uyznuu-4 has negative object offset scale. Try mirroring + resetting scale.',
   'gate-vqjuvg-0-leaf2-base_link-R-lower has negative object offset scale. Try mirroring + resetting scale.',
   'L-pot_plant-ibvdwh-1 has different material. Match materials on each instance.',
   'L-pot_plant-ibvdwh-2 has different material. Match materials on each instance.',
   'bush-uyznuu-2 has different 

In [5]:
total = len(jsons)
failed = {name for name, x in jsons.items() if not x["success"]}
total, len(failed)

(1931, 52)

In [6]:
print("dvc unprotect", " ".join(f"cad/objects/{x}/processed.max" for x in sorted(failed)))

dvc unprotect cad/objects/cleaner-ed/processed.max cad/objects/duster-rm/processed.max cad/objects/duvet_set-pa/processed.max cad/objects/evergreen_tree-is/processed.max cad/objects/frame-ow/processed.max cad/objects/garden_glove-ld/processed.max cad/objects/gin-ie/processed.max cad/objects/hammock-ht/processed.max cad/objects/hamster_wheel-fd/processed.max cad/objects/heat_gun-ty/processed.max cad/objects/ice_skates-oq/processed.max cad/objects/jelly_beans-in/processed.max cad/objects/kabobs-nc/processed.max cad/objects/kielbasa-ks/processed.max cad/objects/kiosk-zx/processed.max cad/objects/leaves-ab/processed.max cad/objects/leek-f0/processed.max cad/objects/lily-yz/processed.max cad/objects/linseed_oil-by/processed.max cad/objects/mallet-i1/processed.max cad/objects/pallet-ef/processed.max cad/objects/pecans-5k/processed.max cad/objects/pencil_case-ne/processed.max cad/objects/pencil_holder-la/processed.max cad/objects/pipe-ns/processed.max cad/objects/plastic_cup-dh/processed.max 

In [5]:
import re
error_templates = ["Category (.*) for object .* does not exist on spreadsheet.", "(.*) has too many vertices: .* > 100000", "(.*) has bad name.", ".*(http.client.RemoteDisconnected|google.auth.exceptions.TransportError).*", "(.*) has negative object offset scale.*", "(.*) has scale that is not 1. Reset scale.", ".*No instance ID 0 instance of (.*)", "(.*) has disallowed type.*"]
error_matches = [[] for _ in error_templates]
error_counts = [0 for _ in error_templates]
unknown_error_files = []
for name, f in jsons.items():
    unknown_error = False
    for error in f["errors"]:
        for i, tmpl in enumerate(error_templates):
            match = re.fullmatch(tmpl, error, re.S)
            if match is not None:
                error_matches[i].append(match)
                error_counts[i] += 1
                break
        else:
            print(error)
            unknown_error = True
    if unknown_error:
        unknown_error_files.append(name)
        
print("\nCounts")
for cnt, tmpl in zip(error_counts, error_templates):
    print(f"{tmpl}: {cnt}")

Volumetric togglebutton meta link blender-dhfqid-0-base_link-Mtogglebutton should be one of {<GeometryClass<Box>>, <GeometryClass<Cone>>, <GeometryClass<Sphere>>, <GeometryClass<Cylinder>>} instead of Dummy
blender-dhfqid-0-base_link meta type togglebutton ID 0 has non-continuous subids ['0', '0']
camera-zcnxuz-0-Mattachment-cameratripod is an invalid meta link under camera-zcnxuz-0.
L-kielbasa-bzlsuh-8 has different pivot offset position (by [131.34037035 -10.50885331   5.98779923]). Match pivots on each instance.
L-kielbasa-bzlsuh-8 has different shear. Match scaling axes on each instance.
L-kielbasa-bzlsuh-7 has different pivot offset position (by [-83.94394554   5.09842162   5.44431026]). Match pivots on each instance.
L-kielbasa-bzlsuh-7 has different shear. Match scaling axes on each instance.
L-kielbasa-bzlsuh-6 has different pivot offset position (by [83.36771654 -6.86039484 -6.80965074]). Match pivots on each instance.
L-kielbasa-bzlsuh-6 has different shear. Match scaling axe

In [6]:
import os
print("dvc repro -f", " ".join("sanitycheck@objects/" + x for x in failed))

dvc repro -f sanitycheck@objects/pencil_case-ne sanitycheck@objects/coffee_beans-aa sanitycheck@objects/cleaner-ed sanitycheck@objects/spices-sr sanitycheck@objects/bedsheet-pj sanitycheck@objects/chili_pepper-ad sanitycheck@objects/pin-gl sanitycheck@objects/pastry-hz sanitycheck@objects/vanilla-yz sanitycheck@objects/bratwurst-lt sanitycheck@objects/organizer-te sanitycheck@objects/french_fries-jh sanitycheck@objects/wallpaper-ll sanitycheck@objects/pillowcase-ye sanitycheck@objects/tote-se sanitycheck@objects/duster-rm sanitycheck@objects/can-yd sanitycheck@objects/cushion-kh sanitycheck@objects/lily-yz sanitycheck@objects/dog_food-df sanitycheck@objects/jelly_beans-in sanitycheck@objects/legacy_washer-mrgspe sanitycheck@objects/console_game-cm sanitycheck@objects/evergreen_tree-is sanitycheck@objects/kielbasa-ks sanitycheck@objects/garden_glove-ld sanitycheck@objects/hammock-ht sanitycheck@objects/quilt-hh sanitycheck@objects/kiosk-zx sanitycheck@objects/poster-k4 sanitycheck@objec

In [7]:
import os
print("dvc unprotect", " ".join(os.path.join(r"cad\objects", x, r"processed.max") for x in unknown_error_files))

dvc unprotect cad\objects\dahlias-po\processed.max cad\objects\legacy_sofa-xhxdqf\processed.max cad\objects\legacy_washer-mrgspe\processed.max cad\objects\linseed_oil-by\processed.max cad\objects\pecans-5k\processed.max cad\objects\pipe-ns\processed.max cad\objects\ramen-va\processed.max


In [10]:
{err: examples for err, examples in zip(error_templates, error_counts)}

{'Category (.*) for object .* does not exist on spreadsheet.': 1,
 '(.*) has too many vertices: .* > 100000': 0,
 '(.*) has bad name.': 10,
 '.*(http.client.RemoteDisconnected|google.auth.exceptions.TransportError).*': 11,
 '(.*) has negative object offset scale.*': 77,
 '(.*) has scale that is not 1. Reset scale.': 661,
 '.*No instance ID 0 instance of (.*)': 6,
 '(.*) has disallowed type.*': 126}

In [11]:
movables = list(glob.glob(r"D:\ig_pipeline\cad\*\*\processed_backup.max"))

In [12]:
import time
import itertools
def grouper(iterator, n: int):
    while chunk := list(itertools.islice(iterator, n)):
        yield chunk

unprotectables = [x.replace("_backup", "") for x in movables]
for g in grouper(iter(unprotectables), 20):
    print(f"dvc add {' '.join(g)}")

dvc add D:\ig_pipeline\cad\objects\acetone-jc\processed.max D:\ig_pipeline\cad\objects\acid-ub\processed.max D:\ig_pipeline\cad\objects\adhesive-io\processed.max D:\ig_pipeline\cad\objects\adhesive_strip-vv\processed.max D:\ig_pipeline\cad\objects\air_conditioner-ep\processed.max D:\ig_pipeline\cad\objects\air_filter-ls\processed.max D:\ig_pipeline\cad\objects\air_freshner-yc\processed.max D:\ig_pipeline\cad\objects\alcohol-ff\processed.max D:\ig_pipeline\cad\objects\alfredo_sauce-zq\processed.max D:\ig_pipeline\cad\objects\algae-tr\processed.max D:\ig_pipeline\cad\objects\allen_wrench-bd\processed.max D:\ig_pipeline\cad\objects\allspice-vw\processed.max D:\ig_pipeline\cad\objects\almond-ab\processed.max D:\ig_pipeline\cad\objects\almond_milk-go\processed.max D:\ig_pipeline\cad\objects\aluminium_foil-xr\processed.max D:\ig_pipeline\cad\objects\ammonia-sd\processed.max D:\ig_pipeline\cad\objects\antihistamines-mr\processed.max D:\ig_pipeline\cad\objects\antlers-mq\processed.max D:\ig_pi

In [13]:
x

'D:\\ig_pipeline\\cad\\objects\\zipper-hx\\artifacts\\sanitycheck.json'

In [51]:
# Check object counts
objectlistfiles = list(glob.glob(r"D:\ig_pipeline\cad\objects\*\artifacts\object_list.json"))
objectlists = {}
for x in tqdm(objectlistfiles):
    with open(x, "r") as f:
        name = pathlib.Path(x).parts[-4] + "/" + pathlib.Path(x).parts[-3]
        objectlists[name] = json.load(f)

  0%|          | 0/1928 [00:00<?, ?it/s]

In [52]:
totalobj = len(objectlists)
failedobj = {name for name, x in objectlists.items() if not x["success"]}
totalobj, len(failedobj)

(1928, 0)

In [6]:
print("dvc unprotect", " ".join(f"cad/{x}/artifacts/object_list.json" for x in failedobj))

NameError: name 'failedobj' is not defined

In [54]:
data = {name: len(x["provided_objects"]) for name, x in objectlists.items()}

In [55]:
sorted(data.items(), key=lambda x: -x[1])

[('objects/pecans-5k', 52),
 ('objects/poster-k4', 49),
 ('objects/bedsheet-pj', 20),
 ('objects/letter-fe', 20),
 ('objects/pastry-hz', 20),
 ('objects/spices-pa', 20),
 ('objects/sunflower-ph', 19),
 ('objects/moth_ball-pg', 18),
 ('objects/quiche-pc', 18),
 ('objects/kabobs-nc', 16),
 ('objects/kielbasa-ks', 16),
 ('objects/pencil_case-ne', 15),
 ('objects/lily-yz', 14),
 ('objects/salad-ur', 14),
 ('objects/shrimp-oq', 13),
 ('objects/cilantro-lx', 12),
 ('objects/crayons-ew', 12),
 ('objects/fruit_tree-rl', 12),
 ('objects/marigold-i7', 11),
 ('objects/peanuts-7y', 11),
 ('objects/bookend-nu', 10),
 ('objects/brisket-fi', 10),
 ('objects/fireplace-pf', 10),
 ('objects/fire_wood-mr', 10),
 ('objects/french_fries-jh', 10),
 ('objects/log-da', 10),
 ('objects/wood_chips-oh', 10),
 ('objects/baking_powder-ll', 9),
 ('objects/chalk-ic', 9),
 ('objects/dried_herb-sl', 9),
 ('objects/margarine-vs', 9),
 ('objects/pea_pods-hg', 9),
 ('objects/wipes-bf', 9),
 ('objects/creme_brulee-eu', 8)

In [74]:
# Get a list of object files that passed sanitycheck, passed object list, and have fewer than 20 objs in them
sanity_check_failures = {name for name, x in jsons.items() if not x["success"]}
obj_list_successes = {name for name, x in objectlists.items() if x["success"]}
usable_object_files = sorted(obj_list_successes - sanity_check_failures)

In [57]:
print("\n".join((f'"{x}",' for x in usable_object_files)))

"objects/acetone-jc",
"objects/acid-ub",
"objects/adhesive-io",
"objects/adhesive_strip-vv",
"objects/air_conditioner-ep",
"objects/air_filter-ls",
"objects/air_freshner-yc",
"objects/alcohol-ff",
"objects/alfredo_sauce-zq",
"objects/algae-tr",
"objects/allen_wrench-bd",
"objects/allspice-vw",
"objects/almond-ab",
"objects/almond_milk-go",
"objects/aluminium_foil-xr",
"objects/ammonia-sd",
"objects/antihistamines-mr",
"objects/antlers-mq",
"objects/apple_cider-jf",
"objects/apple_pie-bk",
"objects/applesauce-dt",
"objects/apron-iu",
"objects/arepa-od",
"objects/arugula-lx",
"objects/ashtray-gr",
"objects/asparagus-ew",
"objects/aspirin-sr",
"objects/avocado-hz",
"objects/axe-yy",
"objects/baby_bottle-hv",
"objects/baby_monitor-fu",
"objects/bacon-zi",
"objects/bait-ec",
"objects/baking_pan-re",
"objects/baking_powder-ll",
"objects/baking_sheet-gz",
"objects/baking_soda-ni",
"objects/balloon-km",
"objects/balsamic_vinegar-cm",
"objects/bandage-at",
"objects/bandana-ko",
"objects/bar_car

In [75]:
def check_usable():
    em_files = [pathlib.Path(r"D:\ig_pipeline\cad") / f / r"artifacts\export_meshes.json" for f in usable_object_files]
    em_jsons = {}
    for x in tqdm([x for x in em_files if x.exists()]):        
        with open(x, "r") as f:
            name = x.parts[-3]
            em_jsons[name] = json.load(f)
            
    total = len(em_jsons)
    failed = {name for name, x in em_jsons.items() if not x["success"]}
    print(total, len(failed))
    
check_usable()

  0%|          | 0/1837 [00:00<?, ?it/s]

1837 7
